## 부동산 실거래가 Crawling ㄱㄱ

In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
import re
import time
import os
from dotenv import load_dotenv # pip install python-dotenv


# 시작연도
startYear = 2022        # 2023 입력 시, 2023 ~ 2025 년 데이터 수집


def get_seoul_real_estate_data(start_year, end_year = None):
    # start_year부터 end_year(기본: 올해)까지의 데이터를 모두 수집하여 합칩니다.
    
    if end_year is None:
        end_year = datetime.now().year

    all_data = []
    for year in range(start_year, end_year + 1):
        print(f"\n📅 {year}년 데이터 수집 중...")
        # 기존 연도 파라미터를 year로 대체
        # 1 접수연도
        RCPT_YR = year          # 현재 2025년 데이터 -> 연도 변환 하면 됨
        var1 = RCPT_YR

        # 2 자치구코드
        CGG_CD = '%20'
        var2 = CGG_CD

        # 3 자치구명
        CGG_NM = '%20'          # 서울시 행정구 입력
        var3 = CGG_NM

        # 4 법정동코드
        STDG_CD = '%20'
        var4 = STDG_CD

        # 5 지번구분
        LOTNO_SE = '%20'
        var5 = LOTNO_SE

        # 6 지번구분명
        LOTNO_SE_NM = '%20'
        var6 = LOTNO_SE_NM

        # 7 본번
        MNO = '%20'
        var7 = MNO

        # 8 부번
        SNO = '%20'
        var8 = SNO

        # 9 건물명
        BLDG_NM = '%20'
        var9 = BLDG_NM

        # 10 계약일
        CTRT_DAY = '%20'
        var10 = CTRT_DAY

        # 11 건물용도 			# [아파트/단독다가구/연립다세대/오피스텔] 택 1
        BLDG_USG = '아파트' 
        var11 = BLDG_USG

        load_dotenv()
        API_KEY = os.getenv("API_KEY")
        base_url = f'http://openapi.seoul.go.kr:8088/{API_KEY}/xml/tbLnOpendataRtmsV'
    
        try:
            # 1. 전체 데이터 개수 확인
            count_url = f'{base_url}/1/1/{var1}/{var2}/{var3}/{var4}/{var5}/{var6}/{var7}/{var8}/{var9}/{var10}/{var11}'
            count_response = requests.get(count_url)
            root = ET.fromstring(count_response.content)
            
            # 에러 체크
            result = root.find('RESULT')
            if result is not None:
                code = result.find('CODE')
                if code is not None and code.text != 'INFO-000':
                    message = result.find('MESSAGE')
                    print(f"🚨 API 오류 : {code.text} - {message.text if message is not None else '알 수 없는 오류'}")
                    return None
            
            total_count = int(root.find('list_total_count').text)
            print(f"📌 {year}년 전체 데이터 개수 : {total_count}건")
            
            # 2. 1000건씩 나누어서 데이터 가져오기
            max_per_request = 1000  # API 제한사항
            
            for start_idx in range(1, total_count + 1, max_per_request):
                end_idx = min(start_idx + max_per_request - 1, total_count)
                
                print(f"📥 데이터 수집 중 : {start_idx} ~ {end_idx}번째 ({len(range(start_idx, end_idx + 1))}건)")
                
                data_url = f'{base_url}/{start_idx}/{end_idx}/{var1}/{var2}/{var3}/{var4}/{var5}/{var6}/{var7}/{var8}/{var9}/{var10}/{var11}'
                
                # 3. 각 배치 데이터 요청
                response = requests.get(data_url, timeout = 30)
                response.raise_for_status()
                
                # XML 파싱하여 데이터 추출
                batch_root = ET.fromstring(response.content)
                
                # 에러 체크
                batch_result = batch_root.find('RESULT')
                if batch_result is not None:
                    batch_code = batch_result.find('CODE')
                    if batch_code is not None and batch_code.text != 'INFO-000':
                        batch_message = batch_result.find('MESSAGE')
                        print(f"⚠️ 배치 {start_idx} ~ {end_idx} 오류: {batch_code.text} - {batch_message.text if batch_message is not None else '알 수 없는 오류'}")
                        continue
                
                # row 데이터 추출
                for row in batch_root.findall('row'):
                    row_data = {}
                    for element in row:
                        tag_name = element.tag
                        tag_value = element.text if element.text else ''
                        row_data[tag_name] = tag_value
                    all_data.append(row_data)
                
                # API 호출 간격 (서버 부하 방지)
                time.sleep(0.1)
            
            print(f"✅ 총 수집된 데이터: {len(all_data)}건")
        except Exception as e:
            print(f"{year}년 데이터 수집 중 오류 : {e}")
            continue # 다음 연도로 넘어감
            
    # XML 없이 바로 DataFrame으로 반환            
    df = pd.DataFrame(all_data)
    return df


def preprocess_dataframe(df):
    # DataFrame 전처리 및 추가 컬럼 생성
    if df.empty:
        return df
    
    # 숫자형 데이터 변환
    numeric_columns = ['RCPT_YR', 'CGG_CD', 'STDG_CD', 'MNO', 'SNO', 
                      'THING_AMT', 'ARCH_AREA', 'LAND_AREA', 'FLR', 'ARCH_YR']
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # 날짜 변환
    date_columns = ['CTRT_DAY', 'RTRCN_DAY']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y%m%d', errors='coerce')
    
    # 건축연대 그룹화
    if 'ARCH_YR' in df.columns:
        df['ARCH_DECADE'] = df['ARCH_YR'].apply(
            lambda x: f"{(x // 10) * 10}년대" if pd.notna(x) else "미상"
        )
    
    # 평수 계산 (건축면적 기준)
    if 'ARCH_AREA' in df.columns:
        df['PYEONG'] = df['ARCH_AREA'] * 0.3025  # ㎡ → 평 변환
        df['PYEONG_GROUP'] = df['PYEONG'].apply(
            lambda x: f"{int(x // 10) * 10}평형대" if pd.notna(x) and x > 0 else "미상"
        )
    
    # 거래금액을 억원 단위로 변환
    if 'THING_AMT' in df.columns:
        df['PRICE_EUK'] = df['THING_AMT'] / 10000
    
    # 평당가(만원/평) 추가
    if 'THING_AMT' in df.columns and 'PYEONG' in df.columns:
        df['PRICE_PER_PYEONG'] = df.apply(
            lambda row: row['THING_AMT'] / row['PYEONG'] if pd.notna(row['THING_AMT']) and pd.notna(row['PYEONG']) and row['PYEONG'] > 0 else None,
            axis=1
        )

    # 데이터 정렬 (최신 계약일 순)
    if 'CTRT_DAY' in df.columns:
        df = df.sort_values('CTRT_DAY', ascending = False)
    
    return df


def save_to_csv(df, filename = None):
    # DataFrame을 CSV 파일로 저장
    if df.empty:
        print("저장할 데이터가 없습니다.")
        return
    
    # data 디렉토리 생성
    import os
    os.makedirs('data', exist_ok = True)
    
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'data/{timestamp}_seoul_real_estate.csv'
    
    try:
        df.to_csv(filename, index = False, encoding = 'utf-8-sig')
        print(f"✅ 데이터 저장 완료 : {filename}")
        print(f"📊 총 {len(df)}건의 데이터")
        
    except Exception as e:
        print(f"파일 저장 오류 : {e}")




def main():
    print("🚀 서울시 부동산 실거래가 데이터 수집 시작")
    print("=" * 50)
    
    # 1. XML 데이터 수집
    df = get_seoul_real_estate_data(start_year = startYear)
    
    if df is None or df.empty:
        print("❌ 데이터 수집 실패")
        return
    
    # 2. 데이터 전처리
    print("\n🔧 데이터 전처리 중...")
    df = preprocess_dataframe(df)
    
    # 3. DataFrame 정보 출력
    print("\n📋 DataFrame 정보:")
    print(f"행 수: {len(df)}")
    print(f"열 수: {len(df.columns)}")
    print("\n컬럼 목록:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col}")
    
    # 4. 샘플 데이터 출력
    print("\n🔍 샘플 데이터 (상위 3개):")
    print(df.head(3)[['CGG_NM', 'STDG_NM', 'BLDG_NM', 'CTRT_DAY', 'THING_AMT', 'ARCH_AREA']].to_string())
    
    # 기본 통계 출력
    if 'THING_AMT' in df.columns:
        print(f"\n💰 평균 거래가 : {df['THING_AMT'].mean():,.0f}만원")
        print(f"💰 최고 거래가 : {df['THING_AMT'].max():,.0f}만원")
    
    # 평당가 통계 추가
    if 'PRICE_PER_PYEONG' in df.columns:
        print(f"\n🏠 평균 평당가 : {df['PRICE_PER_PYEONG'].mean():,.2f}만원/평")
        print(f"🏠 최고 평당가 : {df['PRICE_PER_PYEONG'].max():,.2f}만원/평")
    
    if 'ARCH_DECADE' in df.columns:
        print("\n🏢 건축연대별 분포 :")
        print(df['ARCH_DECADE'].value_counts().head())
    
    if 'PYEONG_GROUP' in df.columns:
        print("\n📐 평수대별 분포 :")
        print(df['PYEONG_GROUP'].value_counts().head())
    
    print("\n✅ 작업 완료!")

    return df

if __name__ == "__main__":
    df = main()

🚀 서울시 부동산 실거래가 데이터 수집 시작

📅 2022년 데이터 수집 중...
📌 2022년 전체 데이터 개수 : 13031건
📥 데이터 수집 중 : 1 ~ 1000번째 (1000건)
📥 데이터 수집 중 : 1001 ~ 2000번째 (1000건)
📥 데이터 수집 중 : 2001 ~ 3000번째 (1000건)
📥 데이터 수집 중 : 3001 ~ 4000번째 (1000건)
📥 데이터 수집 중 : 4001 ~ 5000번째 (1000건)
📥 데이터 수집 중 : 5001 ~ 6000번째 (1000건)
📥 데이터 수집 중 : 6001 ~ 7000번째 (1000건)
📥 데이터 수집 중 : 7001 ~ 8000번째 (1000건)
📥 데이터 수집 중 : 8001 ~ 9000번째 (1000건)
📥 데이터 수집 중 : 9001 ~ 10000번째 (1000건)
📥 데이터 수집 중 : 10001 ~ 11000번째 (1000건)
📥 데이터 수집 중 : 11001 ~ 12000번째 (1000건)
📥 데이터 수집 중 : 12001 ~ 13000번째 (1000건)
📥 데이터 수집 중 : 13001 ~ 13031번째 (31건)
✅ 총 수집된 데이터: 13031건

📅 2023년 데이터 수집 중...
📌 2023년 전체 데이터 개수 : 35640건
📥 데이터 수집 중 : 1 ~ 1000번째 (1000건)
📥 데이터 수집 중 : 1001 ~ 2000번째 (1000건)
📥 데이터 수집 중 : 2001 ~ 3000번째 (1000건)
📥 데이터 수집 중 : 3001 ~ 4000번째 (1000건)
📥 데이터 수집 중 : 4001 ~ 5000번째 (1000건)
📥 데이터 수집 중 : 5001 ~ 6000번째 (1000건)
📥 데이터 수집 중 : 6001 ~ 7000번째 (1000건)
📥 데이터 수집 중 : 7001 ~ 8000번째 (1000건)
📥 데이터 수집 중 : 8001 ~ 9000번째 (1000건)
📥 데이터 수집 중 : 9001 ~ 10000번째 (1000건)
📥 데이터 수집 중 : 1000

In [2]:
# 5. CSV 저장
print("\n💾 CSV 파일 저장 중...")
save_to_csv(df)


💾 CSV 파일 저장 중...
✅ 데이터 저장 완료 : data/20250605_234254_seoul_real_estate.csv
📊 총 137266건의 데이터
